# DEA Coastlines generation using command line tools <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

This notebook demonstrates how to run a DEA Coastlines analysis using command line tools.

Set working directory to top level of repo:

In [ ]:
cd ..

### Run DEA Coastlines analysis
#### Run tidally-constrained raster generation

In [ ]:
!python dea_coastlines/raster.py --study_area 4208 --raster_version tide_func --start_year 1987 --end_year 2021

#### Run vector annual shoreline and rates of change statistics generation

In [ ]:
!python dea_coastlines/vector.py --study_area 4208 --raster_version tide_func --vector_version tide_func_fixed --baseline_year 2020

#### Run continental-scale layer generation
Default hotspot values are: 
* 500
* 2500 (or `True`)
* 5000

In [ ]:
!python dea_coastlines/continental.py --vector_version tide_func_fixed --shorelines True --ratesofchange True --hotspots True